In [1]:
import gurobipy as gp
from gurobipy import GRB
import pprint

In [2]:
driver_data_france = {
    "VER": {"cost": 30.5, "practise_position": 1, "team": "REDB"},
    "HAM": {"cost": 39.1, "practise_position": 4, "team": "MERC"},
    "RUS": {"cost": 24.4, "practise_position": 6, "team": "MERC"},
    "LEC": {"cost": 18.8, "practise_position": 3, "team": "FERR"},
    "PER": {"cost": 18.3, "practise_position": 5, "team": "REBB"},
    "SAI": {"cost": 17.3, "practise_position": 2, "team": "FERR"},    
    "NOR": {"cost": 15.9, "practise_position": 9, "team": "MCLA"},
    "RIC": {"cost": 13.5, "practise_position": 11, "team": "MCLA"},
    "GAS": {"cost": 12.9, "practise_position": 13, "team": "ATRB"},
    "ALO": {"cost": 12.6, "practise_position": 7, "team": "ALPI"},
    "OCO": {"cost": 12.3, "practise_position": 17, "team": "ALPI"},
    "VET": {"cost": 11.4, "practise_position": 20, "team": "ASTM"},    
    "BOT": {"cost": 9.6, "practise_position": 14, "team": "ALFA"},
    "STR": {"cost": 9.2, "practise_position": 18, "team": "ASTM"},
    "TSU": {"cost": 8.3, "practise_position": 10, "team": "ATRB"},
    "GUA": {"cost": 8.2, "practise_position": 15, "team": "ALFA"},
    "ALB": {"cost": 7.5, "practise_position": 8, "team": "WILL"},    
    "LAT": {"cost": 6.7, "practise_position": 12, "team": "WILL"},
    "SCH": {"cost": 6.3, "practise_position": 19, "team": "HAAS"},
    "MAG": {"cost": 6.1, "practise_position": 16, "team": "HAAS"}
}

driver_data_hungary_practise2 = {
    "VER": {"cost": 30.4, "practise_position": 4, "team": "REDB"},
    "HAM": {"cost": 30.1, "practise_position": 11, "team": "MERC"},
    "RUS": {"cost": 24.0, "practise_position": 8, "team": "MERC"},
    "LEC": {"cost": 18.8, "practise_position": 1, "team": "FERR"},
    "PER": {"cost": 18.2, "practise_position": 9, "team": "REBB"},
    "SAI": {"cost": 17.2, "practise_position": 3, "team": "FERR"},    
    "NOR": {"cost": 15.9, "practise_position": 2, "team": "MCLA"},
    "RIC": {"cost": 14.0, "practise_position": 5, "team": "MCLA"},
    "GAS": {"cost": 12.9, "practise_position": 15, "team": "ATRB"},
    "ALO": {"cost": 12.6, "practise_position": 6, "team": "ALPI"},
    "OCO": {"cost": 12.3, "practise_position": 13, "team": "ALPI"},
    "VET": {"cost": 11.4, "practise_position": 7, "team": "ASTM"},    
    "BOT": {"cost": 9.6, "practise_position": 10, "team": "ALFA"},
    "STR": {"cost": 9.2, "practise_position": 14, "team": "ASTM"},
    "TSU": {"cost": 8.3, "practise_position": 19, "team": "ATRB"},
    "GUA": {"cost": 8.2, "practise_position": 12, "team": "ALFA"},
    "ALB": {"cost": 7.6, "practise_position": 20, "team": "WILL"},    
    "LAT": {"cost": 6.7, "practise_position": 12, "team": "WILL"},
    "SCH": {"cost": 6.3, "practise_position": 17, "team": "HAAS"},
    "MAG": {"cost": 6.1, "practise_position": 16, "team": "HAAS"}
}

driver_data = driver_data_hungary_practise2

In [3]:
# Driver scores
race_scores = {1: 25, 2: 18, 3: 15, 4: 12, 5: 10, 6: 8, 7: 7, 8: 4, 9: 2, 10: 1}
quali_scores = {1: 10, 2: 9, 3: 8, 4: 7, 5: 6, 6: 5, 7: 4, 8: 3, 9: 2, 10: 1}
positional_bonus_scores = {1: 5}
for driver in driver_data:
    data = driver_data[driver]
    pos = data["practise_position"]
    if pos > 20:
        continue
    data["score"] = 1 # Classified for quali 1pts

    # position score
    data["score"] += race_scores.get(pos, 0) 
    data["score"] += quali_scores.get(pos, 0) 
    data["score"] += positional_bonus_scores.get(pos, 0) 
    data["score"] += 1 # Finish Q1
    if pos <= 15: # Finish Q2
        data["score"] += 2 
    if pos <= 10: # Finish Q3
        data["score"] += 3

    # Beat teammate 5pts (3 from quali, 2 from race)
    for other_driver, other_driver_data in driver_data.items():
        if other_driver_data["team"] == data["team"] and other_driver != driver:
            if pos < other_driver_data["practise_position"]:
                data["score"] += 5
    
    data["turbo_score"] = 2 * data["score"]

In [16]:
# Manual score-increase, change scaler if wanted
driver = "VER"
scaler = 1.0
print(driver_data[driver]["score"])
driver_data[driver]["score"] *= scaler
print(driver_data[driver]["score"])

26.0
26.0


In [5]:
team_data = {
    "REDB": {"cost": 32.6}, 
    "MERC": {"cost": 33.7}, 
    "FERR": {"cost": 25.8}, 
    "MCLA": {"cost": 17.5}, 
    "ATRB": {"cost": 10.1}, 
    "ALPI": {"cost": 14.0}, 
    "ALFA": {"cost": 8.5}, 
    "ASTM": {"cost": 11.0}, 
    "WILL": {"cost": 6.5}, 
    "HAAS": {"cost": 6.4}}

for team in team_data:
    for driver, data in driver_data.items():
        if data["team"] == team:
            team_data[team]["score"] = team_data[team].get("score", 0) + driver_data[driver]["score"]
            team_data[team]["turbo_score"] = 0.0

In [15]:
bank = 100.0 # M€
driver_and_team_data = driver_data | team_data
drivers = list(driver_data.keys())
teams = list(team_data.keys())
drivers_and_teams = list(driver_and_team_data.keys())
model_dict = {(driver): [driver_and_team_data[driver]["cost"], driver_and_team_data[driver]["score"]] for driver in driver_and_team_data}

for a in driver_and_team_data:
    print(a, driver_and_team_data[a])

VER {'cost': 30.4, 'practise_position': 4, 'team': 'REDB', 'score': 26.0, 'turbo_score': 52}
HAM {'cost': 30.1, 'practise_position': 11, 'team': 'MERC', 'score': 4, 'turbo_score': 8}
RUS {'cost': 24.0, 'practise_position': 8, 'team': 'MERC', 'score': 19, 'turbo_score': 38}
LEC {'cost': 18.8, 'practise_position': 1, 'team': 'FERR', 'score': 52, 'turbo_score': 104}
PER {'cost': 18.2, 'practise_position': 9, 'team': 'REBB', 'score': 11, 'turbo_score': 22}
SAI {'cost': 17.2, 'practise_position': 3, 'team': 'FERR', 'score': 30, 'turbo_score': 60}
NOR {'cost': 15.9, 'practise_position': 2, 'team': 'MCLA', 'score': 39, 'turbo_score': 78}
RIC {'cost': 14.0, 'practise_position': 5, 'team': 'MCLA', 'score': 23, 'turbo_score': 46}
GAS {'cost': 12.9, 'practise_position': 15, 'team': 'ATRB', 'score': 9, 'turbo_score': 18}
ALO {'cost': 12.6, 'practise_position': 6, 'team': 'ALPI', 'score': 25, 'turbo_score': 50}
OCO {'cost': 12.3, 'practise_position': 13, 'team': 'ALPI', 'score': 4, 'turbo_score': 8

In [7]:
combinations, cost, scores = gp.multidict(model_dict)
m = gp.Model('RAP')

Restricted license - for non-production use only - expires 2023-10-25


In [8]:
# Create decision variables for the RAP model
x = m.addVars(combinations, vtype=GRB.BINARY, name="assign")

g = m.addVars(drivers_and_teams, name="gap")
g_d = m.addVars(drivers, name="driver_gap")
g_t = m.addVars(teams, name="team_gap")

In [9]:
# Create job constraints with gap variable
# Gap variable allowes a driver to not be chosen, at no cost, by setting gap[j] = 1
general_gap = m.addConstrs((x.sum(r, '*') + g[r]  == 1 for r in drivers_and_teams), name='picks')
driver_gap = m.addConstrs((x.sum(d, '*') + g_d[d]  == 1 for d in drivers), name='picks')
team_gap = m.addConstrs((x.sum(t, '*') + g_t[t]  == 1 for t in teams), name='picks')

# x.sum(.-.) means sum of the choices. Chosen = 1, not chosen = 0. 
total_picks = m.addConstrs((g.sum('*', '*') >= len(drivers_and_teams) - 6 for r in drivers_and_teams), name='total_picks')
driver_picks = m.addConstrs((g_d.sum('*', '*') >= len(drivers) - 5 for d in drivers), name='driver_picks')
team_picks = m.addConstrs((g_t.sum('*', '*') >= len(teams) - 1 for t in teams), name='team_picks')

In [10]:
budget = m.addConstr((x.prod(cost) <= bank), name='budget')
m.setObjective(x.prod(scores), GRB.MAXIMIZE)

In [11]:
m.write('RAP.lp')
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 121 rows, 90 columns and 1550 nonzeros
Model fingerprint: 0xda42114c
Variable types: 60 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [2e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 86.0000000
Presolve removed 117 rows and 63 columns
Presolve time: 0.00s
Presolved: 4 rows, 27 columns, 81 nonzeros
Variable types: 0 continuous, 27 integer (27 binary)
Found heuristic solution: objective 244.0000000

Root relaxation: objective 2.480349e+02, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  248.03488    0    1  244.00000  248.03488  1.65%     -  

In [14]:
# Display optimal values of decision variables
tot_cost = 0.0
turboscore = 0.0

for v in m.getVars():
    if v.x > 1e-6 and v.varName[0:3] == "ass":
        print(v.varName, v.x)
        driver_name = v.varName.replace("assign[", "").replace("]", "")
        cost = driver_and_team_data[driver_name]["cost"]
        tot_cost += cost
        
        if len(driver_name) == 3 and cost < 20:
            score = driver_and_team_data[driver_name]["score"]
            if score > turboscore:
                turboscore = score
                turbo_name = driver_name
            
# Display optimal total matching score
print("Turbo driver on", turbo_name)
print('Expected score:', m.objVal + turboscore/2)
print("Total cost:", tot_cost)

assign[LEC] 1.0
assign[NOR] 1.0
assign[RIC] 1.0
assign[ALO] 1.0
assign[VET] 1.0
assign[FERR] 1.0
Turbo driver on LEC
Expected score: 270.0
Total cost: 98.5


In [308]:
# Create gap variables for the RAP model
#g_d_t = m.addVars(drivers_and_teams, name="gap_t_d")

In [309]:
# Create job constraints with gap variable
# Gap variable allowes a driver to not be chosen, at no cost, by setting g[j] = 1
#picks = m.addConstrs((x.sum(r, '*') + g[r]  == 1 for r in drivers_and_teams), name='picks')

# x.sum(.-.) means sum of the choices. Chosen = 1, not chosen = 0. 

#total_picks = m.addConstrs((g_d_t.sum('*', '*') >= len(drivers_and_teams) - 6 for r in drivers_and_teams), name='total_picks')

#nr_of_drivers = m.addConstrs((g_d.sum('*', d) >= 15 for d in drivers), name='nr_of_drivers')
#nr_of_teams = m.addConstrs((g.sum('*', t) >= 9 for t in teams), name='nr_of_teams')

In [311]:
#driver_pick = m.addConstrs((x.sum(d, '*') + g_d[d] == 1 for d in drivers), name='driver_pick')
#total_driver_picks = m.addConstrs((g_d.sum('*', '*') >= len(drivers) - 4 for d in drivers), name='total_driver_picks')

#team_pick = m.addConstrs((x.sum(t, '*') + g_t[t]  == 1 for t in teams), name='team_pick')
#total_team_picks = m.addConstrs((g_t.sum('*', '*') >= len(teams) - 1 for t in teams), name='total_team_picks')

#turbo_pick = m.addConstrs((x_turbo.sum(td, '*') + g_turbo[td]  == 1 for td in drivers), name='turbo_pick')
#total_turbo_drivers = m.addConstrs((g_turbo.sum('*', '*') >= len(drivers) - 1 for d in drivers), name='total_turbo_drivers')
#total_turbo_teams = m.addConstrs((g_turbo.sum('*', '*') >= len(teams) for t in teams), name='total_turbo_teams')

#nr_of_drivers = m.addConstrs((g_d.sum('*', d) >= 15 for d in drivers), name='nr_of_drivers')
#nr_of_teams = m.addConstrs((g_d.sum('*', t) >= 9 for t in teams), name='nr_of_teams')

#budget = m.addConstr((x.prod(cost) <= bank), name='budget')